In [1]:
from common import prepare_val
import torch

device = torch.device('cuda:0') # cuda:5
exp_id = '2025_0328_170504'
CHECKPOINT_PATH = f"../logs/GaussianLSS/{exp_id}/checkpoints/last.ckpt"
overrides = ['visualization.keys=[lane, road_segment, vehicle]', '+data.map_layers=[lane, road_segment]']
model, network, loader, viz, dataset = prepare_val('GaussianLSS', device, CHECKPOINT_PATH, overrides=overrides, mode='split', batch_size=1)

Loaded pretrained weights for efficientnet-b4


/home/hcis-s20/miniconda3/envs/GaussianLSS/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hcis-s20/miniconda3/envs/GaussianLSS/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Initializing nuScenes map...
Dataset length: 49
Loaded checkpoint.


In [2]:
import time

GIF_PATH = 'predictions.gif'
network.eval()
network.to(device)
images = []
with torch.no_grad():
    for i,batch in enumerate(loader):
        print(i,end='\r')
        for k, v in batch.items():
            if isinstance(v, torch.Tensor):
                batch[k] = v.to(device)
            elif isinstance(v, list):
                if isinstance(v[0], torch.Tensor):
                    batch[k] = [i.to(device) for i in v]
            else:
                batch[k] = v
        pred = network(batch)
        vis_result = viz(batch,pred)
        images += vis_result

/home/hcis-s20/miniconda3/envs/GaussianLSS/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
from PIL import Image

frames = [Image.fromarray(f) for f in images]  # or use Image.fromarray(np_array)
frames[0].save(
    GIF_PATH,
    save_all=True,
    append_images=frames[1:],
    duration=750,   # milliseconds
    loop=0          # 0 = infinite loop
)